baseline思路：使用CNN进行定长字符分类；

运行系统要求：Python2/3，内存4G，有无GPU都可以

In [1]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

%pylab inline

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

Populating the interactive namespace from numpy and matplotlib


# 定义读取数据集

In [13]:
class SVHNDataset(Dataset):#继承from torch.utils.data.dataset import Dataset
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label 
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    # All subclasses should overwrite __getitem__(), supporting fetching a data sample for a given key. 
    def __getitem__(self, index):
        #orignial pic is already in RGB style so no need to convert
        #img = Image.open(self.img_path[index]).convert('RGB')
        img = Image.open(self.img_path[index])

        if self.transform is not None:
            img = self.transform(img)
        
        print(self.img_label[index])
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * ['X'] #定长处理方式，不满5个的用X凑,定长为5
        return img, torch.from_numpy(np.array(lbl[:5]))
    
    #Subclasses could also optionally overwrite __len__(), 
    #which is expected to return the size of the dataset by many Sampler implementations 
    #and the default options of DataLoader.
    def __len__(self):
        return len(self.img_path)
    
# A map-style dataset is one that implements the __getitem__() and __len__() protocols, 
# and represents a map from (possibly non-integral) indices/keys to data samples.
# For example, such a dataset, when accessed with dataset[idx], 
# could read the idx-th image and its corresponding label from a folder on the disk.

In [3]:
f = open('train.json')
type(f)

_io.TextIOWrapper

In [4]:
y2 = json.load(f)
type(y2)

dict

In [21]:
y2['000000.png']

{'height': [219, 219],
 'label': [1, 9],
 'left': [246, 323],
 'top': [77, 81],
 'width': [81, 96]}

In [22]:
train_path = glob.glob('train/*.png') #path list of all train imgs
train_path.sort()
train_json = json.load(open('train.json')) 
#open('train.json') 返回 _io.TextIOWrapper
#json.load() 返回一个字典 key是图像的名字 value是对应的标注
train_label = [train_json[x]['label'] for x in train_json]#这里的x是png的名字比如 ‘000000.png’
#print(len(train_path), len(train_label))

In [28]:
train_label[22]

[4, 7]

In [37]:
lbl = np.array(train_label[22], dtype=np.int)
lbl

array([4, 7])

In [38]:
lbl = list(lbl)  + (5 - len(lbl)) * ['X']
lbl

[4, 7, 'X', 'X', 'X']

In [15]:
ds = SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),#why 64 128
                    transforms.RandomCrop((60, 120)), #裁剪之后图像的尺寸
                    transforms.ColorJitter(0.3, 0.3, 0.2), #明暗  对比度 透明度
                    #transforms.RandomRotation(10), 
                    ## rotation will change the meaning of numbers like 6 to 9s
                    ## so we don;t use it 
                    transforms.ToTensor(),# 将图片转换为pytorch 的tesntor
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
                    # 对图像像素进行归一化  标准化参数是根据imagenet数据集抽样算出来的
    ]))
ds

# 定义读取数据dataloader

假设数据存放在`../input`文件夹下，并进行解压。

In [16]:
train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),#why 64 128
                    transforms.RandomCrop((60, 120)), #裁剪之后图像的尺寸
                    transforms.ColorJitter(0.3, 0.3, 0.2), #明暗  对比度 透明度
                    #transforms.RandomRotation(10), 
                    ## rotation will change the meaning of numbers like 6 to 9s
                    ## so we don;t use it 
                    transforms.ToTensor(),# 将255范围变成0到1范围 -》 归一化
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
                    # 对图像像素进行标准化 标准化参数是根据imagenet数据集抽样算出来的
    ])), 
    batch_size=40, 
    shuffle=True, 
    num_workers=10,
)
#print(train_loader)
val_path = glob.glob('val/*.png')
val_path.sort()
val_json = json.load(open('val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=10,
)

10000 10000


In [9]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

# 定义分类模型

这里使用ResNet18的模型进行特征提取

In [17]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()

        #model_conv = models.resnet18(pretrained=True)
        model_conv = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        print("get pretained model successfully!")
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        #print("self.cnn = model_conv")
        self.fc1 = nn.Linear(512, 11)
        self.fc2 = nn.Linear(512, 11)
        self.fc3 = nn.Linear(512, 11)
        self.fc4 = nn.Linear(512, 11)
        self.fc5 = nn.Linear(512, 11)
        #print("self.fc5 = nn.Linear(512, 11)")
        #print(self.fc5)
    
    def forward(self, img):      
        #print("how it is used???")
        #print(img.shape)
        feat = self.cnn(img)
        #print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        return c1, c2, c3, c4, c5

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    
    print("------------begin to train:----------")
    ##print(model)
    for i, (input, target) in enumerate(train_loader): #这里用到了dataset中的循环访问器
        if use_cuda:
            input = input.cuda()
            target = target.cuda()
        
        #print(str(i) + "th iteration: input is " + str(input))
        #print(type(train_loader))
        #print(input.shape)
        c0, c1, c2, c3, c4 = model(input)
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])
        
        optimizer.zero_grad()  #即将梯度初始化为零
        loss.backward() #即反向传播求梯度
        optimizer.step() #更新所有参数
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                if use_cuda:
                    output = np.concatenate([
                        c0.data.cpu().numpy(), 
                        c1.data.cpu().numpy(),
                        c2.data.cpu().numpy(), 
                        c3.data.cpu().numpy(),
                        c4.data.cpu().numpy()], axis=1)
                else:
                    output = np.concatenate([
                        c0.data.numpy(), 
                        c1.data.numpy(),
                        c2.data.numpy(), 
                        c3.data.numpy(),
                        c4.data.numpy()], axis=1)
                
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

# 训练与验证

In [19]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0

use_cuda = False
if use_cuda:
    model = model.cuda()

for epoch in range(10):
    train_loss = train(train_loader, model, criterion, optimizer, epoch)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    
    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print('Val Acc', val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        # print('Find better model in Epoch {0}, saving model.'.format(epoch))
        torch.save(model.state_dict(), './model.pt')

get pretained model successfully!
------------begin to train:----------
[6, 6]
[3, 5]
[1, 9]
[2, 1, 9]
[3]
[1, 1]
[2, 2]
[2, 7]
[1, 0, 4]
[4, 2]
[7]
[2, 3]
[2, 0]
[1, 4, 0]
[2]
[2, 2]
[3, 0]
[1, 0]
[8, 3]
[7, 4]
[1, 5]
[2, 4, 8]
[1, 9]
[1, 4, 6]
[3, 8, 8]
[8]
[2, 3]
[4, 0]
[2]
[1, 5]
[3, 4]
[8]
[1, 1]
[4]
[2, 4]
[2, 6]
[6, 2]
[1, 3, 8]
[2, 0]
[7]
[1, 9, 9]
[4, 5]
[3, 4, 6]
[9, 6, 0]
[2, 1, 5]
[1, 5, 5]
[4, 4]
[1, 3]
[1, 4]
[8, 1]
[5, 8]
[2, 1]
[5, 2]
[1, 2, 1]
[6, 2]
[3, 3, 3]
[2]
[1, 2]
[1, 0]
[4, 6]
[8, 3]
[4]
[2, 2, 7]
[6, 7]
[6, 0]
[2, 3]
[2, 2]
[3, 9]
[1, 8, 0]
[3, 2]
[2]
[2, 2, 1]
[4]
[2]
[1, 5]
[1, 4, 4]
[1, 3, 8]
[1, 3, 5]
[2, 5]
[5, 7]
[9]
[1, 3, 4]
[4, 1]
[6, 5]
[8, 1]
[2, 2]
[2, 5]
[8]
[6, 6]
[4, 0]
[1, 4]
[2, 4]
[2, 3]
[4, 6, 3]
[1, 8]
[2, 0]
[3, 1, 8]
[7]
[1, 4, 0]
[6, 9]
[2, 1, 2]
[1, 4]
[1, 6]
[2, 5]
[2, 3, 0]
[2, 7]
[2, 5]
[3]
[1, 5, 1, 0]
[6, 6]
[5, 8, 0]
[2, 1]
[3, 8]
[1, 0]
[1, 1]
[2, 4, 4]
[4, 4, 5]
[2, 5]
[4, 1]
[3, 2, 3]
[9, 8]
[4, 0]
[3, 7]
[1, 0]
[5, 8]
[3, 0]
[

KeyboardInterrupt: 

# 预测并生成提交文件

In [11]:
test_path = glob.glob('test_a/*.png')
test_path.sort()
#test_json = json.load(open('test_a.json'))
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((70, 140)),
                    # transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=10,
)

40000 40000


In [13]:
# 加载保存的最优模型
model.load_state_dict(torch.load('model.pt'))

test_predict_label = predict(test_loader, model, 1)
print(test_predict_label.shape)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('test_A_sample_submit.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('submit.csv', index=None)

(40000, 55)
